# Setup

First, install the necessary libraries. In a Jupyter cell, execute:

In [ ]:
%pip install fastapi[all] uvicorn

# 1. User authorization

FastAPI Code

In [ ]:
from fastapi import FastAPI, Request

app = FastAPI()

@app.get("/redirect")
async def retrieve_auth_code(request: Request):
    # Get the authorization code from the query parameters
    authorization_code = request.query_params.get("code")
    if authorization_code:
        print(f"Authorization Code: {authorization_code}")
        return {"Authorization Code": authorization_code}
    else:
        print("Authorization code not found!")
    return {"Error": "Authorization code not found!"}

## 1.1. Running FastAPI with Uvicorn

To run FastAPI in a Jupyter Notebook, we'll use Uvicorn directly. It will run in the background while you're working in the notebook.

In [ ]:
import uvicorn
from threading import Thread

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)  # type: ignore

t = Thread(target=run)
t.start()

Run ngrok in port http 8000. We need https server to register in ebay.

In [ ]:
!ngrok http 8000

## 1.2. Open browser

Build the url and open in a browser 

In [ ]:
import os

url = "https://auth.sandbox.ebay.com/oauth2/authorize"
client_id = os.getenv("EBAY_CLIENT_ID")
redirect_uri = os.getenv("EBAY_REDIRECT_URI")  # <== Change in ebay to http://localhost:8000/redirect

print(f"{url}?client_id={client_id}&redirect_uri={redirect_uri}&response_type=code")

# 2. Access token

Get the access token with the authorization code.

In [ ]:
import os
import requests
import base64

# Combine client_id and client_secret with a colon and Base64 encode them
client_id = os.getenv("EBAY_CLIENT_ID")
client_secret = os.getenv("EBAY_CLIENT_SECRET")
credentials = f"{client_id}:{client_secret}"
print(f'Credentials: {credentials}')
encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")
print(f'Encoded Credentials: {encoded_credentials}')

# Set up headers and body for the request
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': f'Basic {encoded_credentials}'
}

redirect_uri = 'Fernando_Morale-Fernando-CiclAI-hvillr'
authorization_code = 'v^1.1#i^1#p^3#f^0#r^1#I^3#t^Ul41Xzg6REE3RERBMjAyQzkzNzNCNkExQjRGODA2NUZFMjk2OURfMF8xI0VeMTI4NA=='  # <== Replace with the actual authorization code you get


body = {
    'grant_type': 'authorization_code',
    'code': authorization_code, 
    'redirect_uri': redirect_uri
}

response = requests.post('https://api.sandbox.ebay.com/identity/v1/oauth2/token', headers=headers, data=body)

# Print the response (contains the access token)
print(response.json())

if 'access_token' in response.json():
    access_token = response.json()['access_token']
    print(f'Access Token: {access_token}')
    expires_in = response.json()['expires_in']
    print(f'Expires In: {expires_in}')
    refresh_token = response.json()['refresh_token']
    print(f'Refresh Token: {refresh_token}')
    refresh_token_expires_in = response.json()['refresh_token_expires_in']
    print(f'Refresh Token Expires In: {refresh_token_expires_in}')
    token_type = response.json()['token_type']
    print(f'Token Type: {token_type}')

# Test

Test it with an url

In [9]:
import requests

# Make a POST to https://api.sandbox.ebay.com/sell/inventory/v1/inventory_item?limit=2&offset=0
# access_token = 'v^1.1#i^1#I^3#f^0#p^3#r^0#t^H4sIAAAAAAAAAOVabWwb5R2v88ayNgwkKFB1WnCrRTSc/ZzvzmdfY7NL4lBDnKQ+p01Thvfc3XPxE5/vrveSxmWCLJoyNE1C2xBoY0LdGNvgQ6W9CTqJaZUGoqgb0sSnvfBhQQikdZtgFXzZy3N2XpxMtLGvGpbmL9Y993/7/d+e5/53YKmn99DKkZUP+kI3dJxdAksdoRC9G/T2dA/e2Nmxr3sXaCAInV06uNS13PnOkAMruiXkkWOZhoP6Fyu64Qi1xVTYsw3BhA52BANWkCO4iiCJuXEhFgGCZZuuqZh6uD87mgrHEwwLgMLHaMAzkFPJqrEus2Cmwkkgqxqt8EmQVEFCQ+S+43goazguNNxUOAZiDAWSFIgXaEYAvABAhGZjs+H+Y8h2sGkQkggIp2vmCjVeu8HWq5sKHQfZLhESTmfFMWlSzI5mJgpD0QZZ6TU/SC50PWfr1Yipov5jUPfQ1dU4NWpB8hQFOU44mq5r2CpUENeNacH8mqsZFcRZGiRlJcYCLqFeF1eOmXYFule3w1/BKqXVSAVkuNitXsujxBvyPFLctasJIiI72u//HfWgjjWM7FQ4MyyemJYy+XC/NDVlmwtYRaqPlGZYJpFM8Ilw2kUOcSGyixbEFqzCNVV1eWuO3qZrxDRU7LvN6Z8w3WFE7EZbvRMTuAbvEKJJY9IWNde3qZGOW/ciw8/6Ya3H0XNLhh9ZVCGu6K9dXjsG60mxmQbXKy14BfAcYBkW0hxPK+y2tPBrvaXUSPvREaemor4tSIZVqgLtMnItHSqIUoh7vQqysSownBZjSGlTajypUWxS0yiZU+MUrSEEEJJlJZn4/8oQ17Wx7LloI0u236jBTIUlxbTQlKljpRreTlLrO2s5seikwiXXtYRo9PTp05HTTMS056IxAOjoTG5cUkqoAsMbtPjaxBSuZYdC2jGhF9yqRaxZJMlHlBtz4TRjq1PQdqvDXpVcS0jXyd96Am+xML199SOgjuiY+KFAFLUX0iOm4yI1EDQVLWAFFbH6sSPza30LOooOhEw357CRQ27J/PixbcHlt4TsaCBspINCt71QNTQWwK03IEBTtRYTCCycs1GtEdVPJO0FWxwZyUwVMsHCKVpWtlLxXCjrKNtm2cqCBM8nrk8AM2unryG/1tsIo79RFyUxPx4IpuV5bdBGtwBzzxiaOseUTpmxQND8E5SAoSa4ZhkZ7bcR5jNj+Yx0pFiYvD8zEQhpHmk2ckoFH2e7laJ4VBwVyS+XK+jlUuVYJT9hmSPlwfwYrQ6fSaoz+fmFkXjF4mmWUxMLzjQ3znizx00nFtUnsJapzp/g9IX77q0eTaUCOUlCio12uv/4tf4/ctDIHIcWDZCTmPzwMIbeoD0VjeWN+2aHDVuRx3P6TDJ7/5lpKUeDYA4otGcZ2PXkLdaqtEiuAoHMzLVdP4vDZJxjOUQnIICqnGR5BtEyjTRNpWUUkwPvwm2GdwzZBjRUkxrBii5mKWl4htKABlQa0ojiOZqGsTgTcNe6jkH2a72tNq6No4d/2V6xnRJP5DITBSlWBEX/kaAo3pvPZHKbE6nWEDv+Y3l7IfX5HSIAWjjinyQiilmJmtBzS/5SsWZx/06IorJXJfpVZEdsBFXT0Ks755vzkOPWuXfG5CBdj9THRgRGkxq3MjfBg40Fkq6mXW1F4QZzEzxQUUzPcJtW59f6JnsTCjVP17Cu+1XZCsYG9mZQGlCvulhxWo9jbXZIXOzguZLbrByyVkE24VegC3Wz2XTyE9gpmZblZ6IC7R1Cr9WLppF6gZ5Sm9M2ZyxW6wPzVsFu8JMugfXAUqySaaDAUqCqklNSywHckOMPtwMLqb9+aakOsOH3XKeZ9uCiSkS1odZM5ViwWvFr3S9XFTuWv9U0p7IJchsR+XDnmbqNqdVwGKaLNazUZTie7Cg2tlqol4+Us2FYsGGtWcEK1ttse89IwWaZSMU2UtyiZ+P2ArZ++i7mTBvqiNp+GtfLir0Ig51PSSK12Yy6a7ljaO2AKknHJ/PBRpujaKHdHqrILs5CleYpICOaYuMgRkEaJCiaB4jTaA2xXLB5Z9vN5ul4kgWA43hup7i2LTS8C/yvF8HRrd9ipHfVfvRy6CWwHDrfEQoBHlD0ILirp3O6q3NP2CFbQsQhdSSbixEMtQg5TxlkE7RRpIyqFsR2R08I//4N5cOGr0DOfh7cvvEdSG8nvbvhoxCwf/NON/2p2/piDEiCOM0QxWAWHNi820Xv7brl2++fuP2RK+cf/8bTP+n5Svkzr013nFNB3wZRKNS9q2s5tKv0tWfDoT++dxzecJktnJOf2k+B5+DogBaKfF38+4czd/31nvy3LqXO7xcfyF0c+PLQyr5y7oFfPXHKevDyK7vHPrf48F8ir//81vceOnzB+2eJ/uwv73yHWfnz61/63uonDr3w2JXbDv2YypzSjVf2s3vfmOfTqe9/8tJq8uUnz93x2x89Hzt5+Qqz+ovhB5W700t95d47//Vu6XffvNTxeO7Gty5cvOeZgzfveWr11wPiYy8+3Je/49Fn3n7xiwe++53DFy5+4dVP32ScPfzaD245+e83X3178KYjd79583NP3/qzH06++8SfBpBwXJpPrT7yhwMHHVPb98Ken/7t2fGV3zz5wd5/iAeGyi+9P/DQV1Nv9Z6cf3m5m3m+Hsb/ABmPBkmfIwAA'

headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json',
}

response = requests.get('https://api.sandbox.ebay.com/sell/inventory/v1/inventory_item?limit=2&offset=0', headers=headers)
print(response.json())

{'total': 0, 'size': 0}
